# Classification of fruits using convolutional neural network [GPU]

In [1]:
import pandas as pd
import warnings

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (
    Dense,
    ZeroPadding2D,
    Conv2D,
    MaxPool2D,
    Flatten,
    Dropout,
    BatchNormalization,
)
from tensorflow.keras.optimizers import Adam

pd.options.mode.chained_assignment = None
warnings.filterwarnings('ignore')

In [2]:
def load_train(path):
    datagen = ImageDataGenerator(rescale=1/255)
    datagen_flow = datagen.flow_from_directory(
        path,
        target_size=(150, 150),
        batch_size=16,
        class_mode='sparse',
        seed=12345)

    return datagen_flow

def create_model(input_shape):
    model = Sequential()
    model.add(ZeroPadding2D(padding=(1, 1), input_shape=input_shape))
    model.add(Conv2D(75, (5, 5), activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPool2D((2, 2), strides=2))
    model.add(ZeroPadding2D(padding=(1, 1)))
    model.add(Conv2D(50, (5, 5), activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPool2D((2, 2), strides=2))
    model.add(ZeroPadding2D(padding=(1, 1)))
    model.add(Conv2D(20, (5, 5), activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPool2D((2, 2), strides=2))
    model.add(Flatten())
    model.add(Dense(units=400, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(units=120, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(units=84, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(12, activation='softmax'))
    model.compile(optimizer=Adam(learning_rate=0.001), 
        loss='sparse_categorical_crossentropy', metrics=['acc'])
    return model

def train_model(model, train_data, test_data, batch_size=None, epochs=20,
                steps_per_epoch=None, validation_steps=None):
    model.fit(train_data,
        validation_data=test_data,
        epochs=epochs, batch_size=batch_size,
        steps_per_epoch=steps_per_epoch,
        validation_steps=validation_steps,
        verbose=2, shuffle=True)
    return model

**Neural Network Architecture**

In [3]:
model = create_model(input_shape=(150, 150, 3))
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ zero_padding2d (ZeroPadding2D)  │ (None, 152, 152, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 148, 148, 75)   │         5,700 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 148, 148, 75)   │           300 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 74, 74, 75)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ zero_padding2d_1                │ (None, 76, 76, 75)     │             0 │
│ (ZeroPadding2D)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 72, 72, 50)     │        93,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 72, 72, 50)     │           200 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 36, 36, 50)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ zero_padding2d_2                │ (None, 38, 38, 50)     │             0 │
│ (ZeroPadding2D)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 34, 34, 20)     │        25,020 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 34, 34, 20)     │            80 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 17, 17, 20)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 5780)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 400)            │     2,312,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 400)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 120)            │        48,120 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 120)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 84)             │        10,164 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 84)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 12)             │         1,020 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,496,804 (9.52 MB)

 Trainable params: 2,496,514 (9.52 MB)

 Non-trainable params: 290 (1.13 KB)

**Result of output model**
```
Found 23397 images belonging to 12 classes.
Found 7804 images belonging to 12 classes.

Train for 1463 steps, validate for 488 steps
Epoch 1/20
1463/1463 - 169s - loss: 0.7462 - acc: 0.7608 - val_loss: 1.0486 - val_acc: 0.6898
Epoch 2/20
1463/1463 - 153s - loss: 0.2729 - acc: 0.9118 - val_loss: 0.2730 - val_acc: 0.9138
Epoch 3/20
1463/1463 - 156s - loss: 0.1866 - acc: 0.9402 - val_loss: 0.4099 - val_acc: 0.8845
Epoch 4/20
1463/1463 - 155s - loss: 0.1380 - acc: 0.9601 - val_loss: 0.4125 - val_acc: 0.9036
Epoch 5/20
1463/1463 - 156s - loss: 0.1102 - acc: 0.9680 - val_loss: 1.9078 - val_acc: 0.7034
Epoch 6/20
1463/1463 - 153s - loss: 0.0863 - acc: 0.9760 - val_loss: 0.2797 - val_acc: 0.9207
Epoch 7/20
1463/1463 - 155s - loss: 0.0729 - acc: 0.9810 - val_loss: 0.3628 - val_acc: 0.9085
Epoch 8/20
1463/1463 - 154s - loss: 0.0626 - acc: 0.9846 - val_loss: 0.3079 - val_acc: 0.9357
Epoch 9/20
1463/1463 - 155s - loss: 0.0539 - acc: 0.9856 - val_loss: 0.1991 - val_acc: 0.9637
Epoch 10/20
1463/1463 - 155s - loss: 0.0561 - acc: 0.9860 - val_loss: 0.4209 - val_acc: 0.9402
Epoch 11/20
1463/1463 - 155s - loss: 0.0515 - acc: 0.9881 - val_loss: 0.3157 - val_acc: 0.9450
Epoch 12/20
1463/1463 - 156s - loss: 0.0384 - acc: 0.9911 - val_loss: 0.0923 - val_acc: 0.9786
Epoch 13/20
1463/1463 - 161s - loss: 0.0438 - acc: 0.9907 - val_loss: 0.0842 - val_acc: 0.9783
Epoch 14/20
1463/1463 - 161s - loss: 0.0466 - acc: 0.9889 - val_loss: 0.1561 - val_acc: 0.9645
Epoch 15/20
1463/1463 - 156s - loss: 0.0384 - acc: 0.9926 - val_loss: 0.5260 - val_acc: 0.9466
Epoch 16/20
1463/1463 - 162s - loss: 0.0352 - acc: 0.9927 - val_loss: 0.1025 - val_acc: 0.9818
Epoch 17/20
1463/1463 - 161s - loss: 0.0385 - acc: 0.9925 - val_loss: 0.0868 - val_acc: 0.9799
Epoch 18/20
1463/1463 - 161s - loss: 0.0375 - acc: 0.9924 - val_loss: 0.1877 - val_acc: 0.9695
Epoch 19/20
1463/1463 - 162s - loss: 0.0290 - acc: 0.9936 - val_loss: 0.1122 - val_acc: 0.9763
Epoch 20/20
1463/1463 - 161s - loss: 0.0314 - acc: 0.9935 - val_loss: 0.1601 - val_acc: 0.9635

488/488 - 40s - loss: 0.1601 - acc: 0.9635
```